In [1]:
# Install dependencies
!pip install python-chess networkx stockfish numpy scipy

# Import libraries
import chess
import networkx as nx
import numpy as np
from stockfish import Stockfish
from scipy.sparse.csgraph import laplacian
from scipy.sparse.linalg import eigsh
import random

# Define ChessPosition class
class ChessPosition:
    def __init__(self, fen=chess.STARTING_FEN):
        self.board = chess.Board(fen)
    
    def morphism(self, move):
        """Legal move (morphism) between positions."""
        new_board = self.board.copy()
        new_board.push(move)
        return ChessPosition(new_board.fen())
    
    def __eq__(self, other):
        return self.board.fen() == other.board.fen()

# Define GraphConstructionFunctor
class GraphConstructionFunctor:
    def __init__(self, graph_type="control"):
        self.graph_type = graph_type
    
    def __call__(self, position):
        if self.graph_type == "control":
            return self._build_control_graph(position)
        elif self.graph_type == "threat":
            return self._build_threat_graph(position)
        elif self.graph_type == "strategic":
            return self._build_strategic_graph(position)
        elif self.graph_type == "hybrid":
            return self._build_hybrid_graph(position)
        else:
            raise ValueError("Unsupported graph type")

    def _build_control_graph(self, position):
        G = nx.Graph()
        board = position.board
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                G.add_node(square, piece=piece.symbol())
                for move in board.attacks(square):
                    if board.piece_at(move):
                        G.add_edge(square, move)
        return G

    def _build_threat_graph(self, position):
        G = nx.DiGraph()
        board = position.board
        for square in chess.SQUARES:
            attacker = board.piece_at(square)
            if attacker:
                for victim_sq in board.attacks(square):
                    victim = board.piece_at(victim_sq)
                    if victim and attacker.color != victim.color:
                        G.add_edge(square, victim_sq)
        return G

    def _build_strategic_graph(self, position):
        G = nx.DiGraph()
        # Add milestones (e.g., passed pawns, open files) as nodes
        # Add edges for transitions between milestones
        return G

    def _build_hybrid_graph(self, position):
        control_graph = self._build_control_graph(position)
        threat_graph = self._build_threat_graph(position)
        strategic_graph = self._build_strategic_graph(position)
        G = nx.compose_all([control_graph, threat_graph, strategic_graph])
        return G

# Define EvaluationFunctor
class EvaluationFunctor:
    def __init__(self, stockfish_path):
        self.evaluator = Stockfish(stockfish_path)
    
    def __call__(self, position):
        self.evaluator.set_fen_position(position.board.fen())
        return self.evaluator.get_evaluation()

def algebraic_control_score(graph):
    """Compute algebraic connectivity and Fiedler vector centrality."""
    L = nx.laplacian_matrix(graph).toarray().astype(float)
    eigenvalues, eigenvectors = np.linalg.eigh(L)
    lambda2 = eigenvalues[1]  # Algebraic connectivity
    fiedler_vector = eigenvectors[:, 1]
    centrality = np.max(np.abs(fiedler_vector))  # Critical node
    return lambda2 + centrality

def algebraic_threat_score(graph):
    """Compute spectral radius and eigenvector centrality."""
    A = nx.adjacency_matrix(graph).toarray().astype(float)
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    lambda_max = np.max(eigenvalues)  # Spectral radius
    eigen_centrality = np.abs(eigenvectors[:, np.argmax(eigenvalues)]).max()
    return lambda_max + eigen_centrality

def algebraic_strategic_score(graph):
    """Compute normalized Laplacian's smallest eigenvalue and betweenness."""
    L_norm = nx.normalized_laplacian_matrix(graph).toarray().astype(float)
    eigenvalues, _ = np.linalg.eigh(L_norm)
    lambda1 = eigenvalues[0]  # Smallest eigenvalue
    betweenness = nx.betweenness_centrality(graph)[np.argmax(betweenness.values())]
    return lambda1 + betweenness

def algebraic_hybrid_score(graphs):
    """Compute spectral radius of the tensor product graph."""
    tensor_graph = nx.tensor_product(graphs[0], graphs[1])
    for g in graphs[2:]:
        tensor_graph = nx.tensor_product(tensor_graph, g)
    A = nx.adjacency_matrix(tensor_graph).toarray().astype(float)
    eigenvalues, _ = np.linalg.eigh(A)
    return np.max(eigenvalues)

class AlgebraicControlRefinementFunctor:
    def __init__(self):
        pass
    
    def __call__(self, control_graph):
        return algebraic_control_score(control_graph)
    
class AlgebraicThreatRefinementFunctor:
    def __init__(self):
        pass
    
    def __call__(self, threat_graph):
        return algebraic_threat_score(threat_graph)
    
class AlgebraicStrategicRefinementFunctor:
    def __init__(self):
        pass
    
    def __call__(self, strategic_graph):
        return algebraic_strategic_score(strategic_graph)
    
class AlgebraicHybridRefinementFunctor:
    def __init__(self):
        pass
    
    def __call__(self, graphs):
        return algebraic_hybrid_score(graphs)
    

# Define helper functions
def normalize_eval(eval_result):
    """Normalize Stockfish evaluation to [-1, 1]."""
    if eval_result['type'] == 'cp':
        return eval_result['value'] / 100.0
    else:
        return 1.0 if eval_result['value'] > 0 else -1.0

def generate_random_position(depth=10):
    """Generate a random chess position by playing `depth` random moves."""
    board = chess.Board()
    for _ in range(depth):
        moves = list(board.legal_moves)
        if not moves:
            break
        move = random.choice(moves)
        board.push(move)
    return ChessPosition(board.fen())

def generate_random_positions(num_positions=100, depth=10):
    """Generate a list of random chess positions."""
    return [generate_random_position(depth) for _ in range(num_positions)]

def are_positions_similar(position1, position2, threshold=0.1):
    """Check if two positions are similar using algebraic metrics."""
    # Compute graphs
    control_graph1 = control_functor(position1)
    threat_graph1 = threat_functor(position1)
    strategic_graph1 = strategic_functor(position1)
    hybrid_graph1 = hybrid_functor(position1)
    
    control_graph2 = control_functor(position2)
    threat_graph2 = threat_functor(position2)
    strategic_graph2 = strategic_functor(position2)
    hybrid_graph2 = hybrid_functor(position2)
    
    # Compute refined scores
    control_score1 = control_refinement(control_graph1)
    threat_score1 = threat_refinement(threat_graph1)
    strategic_score1 = strategic_refinement(strategic_graph1)
    hybrid_score1 = hybrid_refinement([control_graph1, threat_graph1, strategic_graph1])
    
    control_score2 = control_refinement(control_graph2)
    threat_score2 = threat_refinement(threat_graph2)
    strategic_score2 = strategic_refinement(strategic_graph2)
    hybrid_score2 = hybrid_refinement([control_graph2, threat_graph2, strategic_graph2])
    
    # Compare scores
    scores1 = [control_score1, threat_score1, strategic_score1, hybrid_score1]
    scores2 = [control_score2, threat_score2, strategic_score2, hybrid_score2]
    scores_close = all(np.isclose(s1, s2, atol=threshold) for s1, s2 in zip(scores1, scores2))
    
    # Compare evaluations
    eval1 = normalize_eval(evaluator(position1))
    eval2 = normalize_eval(evaluator(position2))
    eval_close = abs(eval1 - eval2) < threshold
    
    return scores_close and eval_close

def find_similar_pairs(num_positions=100, depth=10, threshold=0.1):
    """Find pairs of similar positions from a set of random positions."""
    positions = generate_random_positions(num_positions, depth)
    similar_pairs = []
    
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            if are_positions_similar(positions[i], positions[j], threshold):
                similar_pairs.append((positions[i], positions[j]))
    
    return similar_pairs

# Initialize Stockfish
stockfish_path = "D:\\stockfish\\stockfish.exe"  # Update this to your Stockfish executable path
evaluator = EvaluationFunctor(stockfish_path)

# Define functors
control_functor = GraphConstructionFunctor("control")
threat_functor = GraphConstructionFunctor("threat")
strategic_functor = GraphConstructionFunctor("strategic")
hybrid_functor = GraphConstructionFunctor("hybrid")

control_refinement = AlgebraicControlRefinementFunctor()
threat_refinement = AlgebraicThreatRefinementFunctor()
strategic_refinement = AlgebraicStrategicRefinementFunctor()
hybrid_refinement = AlgebraicHybridRefinementFunctor()

# Find similar pairs
similar_pairs = find_similar_pairs(num_positions=100, depth=10, threshold=0.1)

# Output results
print(f"Found {len(similar_pairs)} similar pairs:")
for pair in similar_pairs:
    print(f"Pair:\n{pair[0].board.fen()}\n{pair[1].board.fen()}\n")

NetworkXNotImplemented: not implemented for directed type

In [2]:
!pip install --upgrade networkx[default]

  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0


ERROR: arviz 0.11.2 has requirement typing-extensions<4,>=3.7.4.3, but you'll have typing-extensions 3.7.4.2 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Anaconda\\Lib\\site-packages\\~cipy\\sparse\\_sparsetools.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
!pip install --user networkx[default]